In [1]:
import srtm
from pyproj import Transformer

def quick_test():

    geo_data = srtm.get_data()
    transformer = Transformer.from_crs("EPSG:32634", "EPSG:4326", always_xy=True)

    # Dane wejściowe: Współrzędne Rynku w Krakowie 
    test_x_metry = 424000  
    test_y_metry = 5546000 

    print(f"1. Wejście (Metry): X={test_x_metry}, Y={test_y_metry}")

    lon, lat = transformer.transform(test_x_metry, test_y_metry)
    print(f"2. Wynik konwersji (GPS): Lat={lat:.4f}, Lon={lon:.4f}")

    elev = geo_data.get_elevation(lat, lon)
    print(f"3. Wysokość z SRTM: {elev}")

if __name__ == "__main__":
    quick_test()

1. Wejście (Metry): X=424000, Y=5546000
2. Wynik konwersji (GPS): Lat=50.0614, Lon=19.9382
3. Wysokość z SRTM: 220
